In [3]:
from bs4 import BeautifulSoup
from random import randint
from time import sleep
import re
import pandas as pd
import json
import math
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm_notebook as tqdm


pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", -1)
# pd.set_option("display.float_format", "{:,.2f}".format)
pd.set_option("display.max_info_rows", 200)

In [ ]:
import sephora_scrape as scrape
%reload_ext autoreload
%autoreload 2

In [2]:
# Get url link
sephora_url = "https://www.sephora.com"

We'll be able to scrape all products from the website by doing it per brand.  First, let's get a list of all brands and the corresponding url link using Requests and BeautifulSoup4.

In [5]:
chrome_path = "/Users/valmadrid/Downloads/chromedriver"
sephora_brands_url = "https://www.sephora.com/brands-list"
browser = webdriver.Chrome(executable_path = chrome_path)
browser.get(sephora_brands_url)

scrape.close_country_box(browser)
scrape.close_login_box(browser)
scrape.close_country_box(browser)
sleep(2)
scrape.scroll_to_bottom(browser, 10)
sleep(2)

In [ ]:
a_elems = browser.find_elements_by_xpath("//a")
brand_list=[]
for a in a_elems:
    brand_dict={}
    if (len(a.text)!=0) & (a.get_attribute("href").startswith("https://www.sephora.com/brand/")):
        brand_dict["brand_name"] = str(a.text).strip().replace(" NEW", "")
        brand_dict["brand_link"]= a.get_attribute("href")
        brand_list.append(brand_dict)

In [ ]:
browser.close()

In [57]:
# df_brands = pd.DataFrame(brand_list)
# df_brands.to_csv("brands.csv", index=False)
df_brands = pd.read_csv("brands.csv")
df_brands

,brand_name,brand_link
0,Acqua di Parma,https://www.sephora.com/brand/acqua-di-parma
1,adwoa beauty,https://www.sephora.com/brand/adwoa-beauty
2,AERIN,https://www.sephora.com/brand/aerin-perfume
3,Aether Beauty,https://www.sephora.com/brand/aether-beauty
4,Algenist,https://www.sephora.com/brand/algenist
...,...,...
333,Youth To The People,https://www.sephora.com/brand/youth-to-the-people
334,YUNI,https://www.sephora.com/brand/yuni
335,Yves Saint Laurent,https://www.sephora.com/brand/yves-saint-laurent
336,The 7 Virtues,https://www.sephora.com/brand/7-virtues


In [1]:
def close_country_box(browser):
    try:
        browser.find_element_by_xpath("//*[@id='MediaModal']/button").click()
        return browser
    except:
        return browser


def close_login_box(browser):
    try:
        browser.find_element_by_xpath("//*[@id='modalDialog']/button").click()
        return browser
    except:
        return browser


def view_300(browser):
    try:
        if browser.find_element_by_class_name("css-1qizhe3").text.endswith("View all"):
            browser.find_element_by_class_name("css-1qizhe3").send_keys("View all")
            sleep(1)
            close_country_box(browser)
            close_login_box(browser)
            close_country_box(browser)
            scroll_to_bottom(browser, 5)
            sleep(1)
            close_country_box(browser)
            close_login_box(browser)
            close_country_box(browser)
            sleep(1)
            return browser
    except:
        return browser
        
def next_page(browser):
    try:
        if browser.find_element_by_class_name("css-4ktkov").get_attribute("aria-label")=="Next":
            browser.find_element_by_class_name("css-4ktkov").click()
            sleep(1)
            close_country_box(browser)
            close_login_box(browser)
            close_country_box(browser)
            scroll_to_bottom(browser, 5)
            sleep(1)
            close_country_box(browser)
            close_login_box(browser)
            close_country_box(browser)
            sleep(1)
            return browser
    except:
        return browser

    
def scroll_to_bottom(browser, x=20):
    sleep(1)
    close_country_box(browser)
    close_login_box(browser)
    close_country_box(browser)
    sleep(1)
    elem = browser.find_element_by_tag_name("body")
    no_of_pagedowns = x
    while no_of_pagedowns:
        elem.send_keys(Keys.PAGE_DOWN)
        sleep(0.5)
        close_country_box(browser)
        close_login_box(browser)
        close_country_box(browser)
        no_of_pagedowns -= 1
    return browser

In [51]:
def get_page(url, page):
 
    browser.get(url)
    
    close_country_box(browser)
    close_login_box(browser)
    close_country_box(browser)
    sleep(2)
    
    scroll_to_bottom(browser, 10)
    sleep(3)
    scroll_to_bottom(browser, 10)
    sleep(3)

    view_300(browser)
    
    scroll_to_bottom(browser, 10)
    sleep(3)
    scroll_to_bottom(browser, 10)
    sleep(3)
#     scroll_to_bottom(browser, 10)
#     sleep(3)

#     if page > 1:
#         for i in range(page - 1):
#             next_page(browser)
#             scroll_to_bottom(browser)
#             sleep(3)
    
#     scroll_to_bottom(browser, 10)
#     sleep(5)
    scroll_to_bottom(browser, 10)
    sleep(10)
    
    close_country_box(browser)
    close_login_box(browser)
    close_country_box(browser)
    sleep(2)

    html = browser.execute_script('return document.documentElement.outerHTML')
    soup = BeautifulSoup(html, 'html.parser')
    soup.prettify()

    return soup

In [41]:
def get_products(soup):

    prefix_url = "https://www.sephora.com"
    
    try:
        product_json = soup.find_all(attrs={"type": re.compile("text/json")})
        product_json = json.loads(product_json[0].text)
    except:
        print(filename, " - download unsuccessful")

    product_list = []

    for i in product_json:
        for key, val in i["props"].items():
            if key == "products":
                if type(val) == list:
                    for product in val:

                        product_dict = {}  #each product

                        product_dict["brand_name"] = product["brandName"]

                        product_dict["product_ref"] = product["productId"]

                        product_dict["product_item"] = product["currentSku"][
                            "skuId"]

                        product_dict["product_name"] = product["displayName"]

                        product_dict["product_price"] = product["currentSku"][
                            "listPrice"]

                        product_dict["product_image"] = prefix_url + product[
                            "currentSku"]["skuImages"]["image450"]

                        product_dict[
                            "product_link"] = prefix_url + product["targetUrl"]

                        product_list.append(product_dict)
                        
    if len(product_list)!=0:
        return product_list
    
    else:

        if len(soup.find_all("a", href=re.compile("/product/"))) == 0:
            return product_list

        else:
            for tag in soup.find_all("a", href=re.compile("/product/")):

                product_dict = {}  #each product

                product_dict["brand_name"] = tag.find(
                    attrs={
                        "data-at": re.compile("sku_item_brand")
                    }).get_text(strip=True).replace(" NEW", "")
                try:
                    product_dict["product_ref"] = tag.attrs["data-uid"].split()[0]
                except:
                    product_dict["product_ref"] = np.nan

                try:
                    product_dict["product_item"] = tag.attrs["data-uid"].split(
                    )[-1]
                except:
                    product_dict["product_item"] = np.nan

                try:
                    product_dict["product_name"] = tag.find(
                        attrs={
                            "data-at": re.compile("sku_item_name")
                        }).get_text(strip=True)
                except:
                    product_dict["product_name"] = np.nan

                try:
                    product_dict["product_price"] = tag.find(
                        attrs={
                            "data-at": re.compile("sku_item_price_list")
                        }).get_text(strip=True)
                except:
                    product_dict["product_price"] = np.nan

                try:
                    product_dict["product_image"] = prefix_url + tag.find(
                        attrs={
                            "src": re.compile("/productimages/")
                        }).get("src")
                    if product_dict["product_item"] is np.nan:
                        item_pattern = re.compile("s[0-9]+")
                        product_dict["product_item"] = item_pattern.findall(
                            product_dict["product_image"])[0].strip("s")
                except:
                    product_dict["product_image"] = np.nan

                try:
                    product_dict["product_link"] = prefix_url + tag.attrs["href"]
                    if product_dict["product_ref"] is np.nan:
                        ref_pattern = re.compile("P[0-9]+")
                        product_dict["product_ref"] = ref_pattern.findall(
                            product_dict["product_link"])[0]
                except:
                    product_dict["product_link"] = np.nan

                product_list.append(product_dict)

        return product_list

In [58]:
df_products = pd.DataFrame()

chrome_path = "/Users/valmadrid/Downloads/chromedriver"
browser = webdriver.Chrome(executable_path=chrome_path)

for i in tqdm(range(len(df_brands))):
    product_list = []
    for page in range(1, 5):
        if page == 1:
            full_url = df_brands.brand_link.iloc[i] + "?pageSize=300"
        if page > 1:
            full_url = df_brands.brand_link.iloc[i] + "?pageSize=300&currentPage=" + str(page)
        soup = get_page(full_url, page)
        products = get_products(soup)  #list of product dicts in page 'page'
        product_list.extend(products)
        if len(product_list) == 0:
            #brand has 0 products, then go to next brand
            break
        else:
            #otherwise append the list to the dataframe
            df_products = df_products.append(pd.DataFrame(product_list),
                                             ignore_index=True)
        if len(products) < 300:
            #current page has less than 300 items, then go to next brand
            break
        else:
            #otherwise go to next page
            continue

    sleep(10)
    df_products.to_csv("final_products.csv", index=False)

browser.quit()

In [60]:
df_products

,brand_name,product_ref,product_item,product_name,product_price,product_image,product_link
0,Acqua di Parma,P307801,1417609,Fico di Amalfi,$30.00 - $180.00,https://www.sephora.com/productimages/sku/s1417609-main-Lhero.jpg,https://www.sephora.com/product/blu-mediterraneo-fico-di-amalfi-P307801
1,Acqua di Parma,P443401,2218774,Blu Mediterraneo MINIATURE Set,$66.00,https://www.sephora.com/productimages/sku/s2218774-main-Lhero.jpg,https://www.sephora.com/product/blu-mediterraneo-minature-set-P443401
2,Acqua di Parma,P163604,2044816,Colonia,$66.00 - $170.00,https://www.sephora.com/productimages/sku/s2044816-main-Lhero.jpg,https://www.sephora.com/product/colonia-P163604
3,Acqua di Parma,P307804,1417617,Mirto di Panarea,$30.00 - $180.00,https://www.sephora.com/productimages/sku/s1417617-main-Lhero.jpg,https://www.sephora.com/product/blu-mediterraneo-mirto-di-panarea-P307804
4,Acqua di Parma,P375388,1417567,Arancia di Capri,$30.00 - $180.00,https://www.sephora.com/productimages/sku/s1417567-main-Lhero.jpg,https://www.sephora.com/product/blu-mediterraneo-arancia-di-capri-P375388
...,...,...,...,...,...,...,...
9295,The 7 Virtues,P428703,2047694,Grapefruit Lime Eau de Parfum Rollerball,$29.00,https://www.sephora.com/productimages/sku/s2047694-main-Lhero.jpg?pb=2020-03-sephora-clean-2019,https://www.sephora.com/product/grapefruit-lime-eau-de-parfum-rollerball-P428703
9296,The 7 Virtues,P428712,2047710,Rose Amber Eau de Parfum Rollerball,$29.00,https://www.sephora.com/productimages/sku/s2047710-main-Lhero.jpg?pb=2020-03-sephora-clean-2019,https://www.sephora.com/product/rose-amber-eau-de-parfum-rollerball-P428712
9297,8Greens,P437988,2162444,8G Dietary Supplement,$14.00 - $78.00,https://www.sephora.com/productimages/sku/s2162444-main-Lhero.jpg,https://www.sephora.com/product/8greens-dietary-supplement-P437988
9298,8Greens,P452918,2298891,8Greens Gummies Dietary Supplement,$45.00,https://www.sephora.com/productimages/sku/s2298891-main-Lhero.jpg,https://www.sephora.com/product/8greens-8greens-gummies-dietary-supplement-P452918


In [63]:
df_products.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9300 entries, 0 to 9299
Data columns (total 7 columns):
brand_name       9300 non-null object
product_ref      9300 non-null object
product_item     9300 non-null object
product_name     9300 non-null object
product_price    9300 non-null object
product_image    9300 non-null object
product_link     9300 non-null object
dtypes: object(7)
memory usage: 508.7+ KB


In [64]:
df_products.to_csv("final_products.csv", index=False)